In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import ast
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn import metrics
# note - RF is giving more differential diagnosis than DT
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import re
import os
import pickle
from constants import base_path
import warnings
warnings.filterwarnings("ignore")

In [2]:
diagnosis_df_train = pd.read_csv(f"{base_path}\\input\\release_train_patients")
diagnosis_df_test = pd.read_csv(f"{base_path}\\input\\release_test_patients")

In [3]:
diagnosis_df_train.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025602 entries, 0 to 1025601
Data columns (total 6 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   AGE                     1025602 non-null  int64 
 1   DIFFERENTIAL_DIAGNOSIS  1025602 non-null  object
 2   SEX                     1025602 non-null  object
 3   PATHOLOGY               1025602 non-null  object
 4   EVIDENCES               1025602 non-null  object
 5   INITIAL_EVIDENCE        1025602 non-null  object
dtypes: int64(1), object(5)
memory usage: 46.9+ MB


In [4]:
diagnosis_df_train.PATHOLOGY.value_counts(normalize=True).plot.bar(figsize=(12,3))
plt.ylabel("Normalized Frequency")
plt.title("Pathology Frequency (Train Set)")
plt.savefig(f'{base_path}\\output\\train_pathology_freq.jpg', bbox_inches='tight')
plt.clf()

<Figure size 1200x300 with 0 Axes>

In [5]:
diagnosis_df_test.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134529 entries, 0 to 134528
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   AGE                     134529 non-null  int64 
 1   DIFFERENTIAL_DIAGNOSIS  134529 non-null  object
 2   SEX                     134529 non-null  object
 3   PATHOLOGY               134529 non-null  object
 4   EVIDENCES               134529 non-null  object
 5   INITIAL_EVIDENCE        134529 non-null  object
dtypes: int64(1), object(5)
memory usage: 6.2+ MB


In [6]:
diagnosis_df_test.PATHOLOGY.value_counts(normalize=True).plot.bar(figsize=(12,3))
plt.ylabel("Normalized Frequency")
plt.title("Pathology Frequency (Test Set)")
plt.savefig(f'{base_path}\\output\\test_pathology_freq.jpg', bbox_inches='tight')
plt.clf()

<Figure size 1200x300 with 0 Axes>

In [7]:
with open(f"{base_path}\\input\\release_conditions.json") as f:
  disease_dict = json.load(f)
disease_list = list(disease_dict.keys())

In [8]:
with open(f"{base_path}\\input\\release_evidences.json") as f:
  evidences = json.load(f)
evidences_list = []
evidences_dict = {}
for e in evidences.keys():
  # only binary symptoms and no antecedents
  if (not evidences[e]["possible-values"]) and (not evidences[e]["is_antecedent"]):
    evidences_list.append(e)
    evidences_dict[e] = evidences[e]["question_en"]
evidences_dict["AGE"] = "AGE"
evidences_dict["SEX"] = "SEX"
feature_columns = ["AGE", "SEX"] + evidences_list

In [9]:
def data_proc(df):
    df["binary_evidences"] = df["EVIDENCES"].apply(lambda x: [d for d in ast.literal_eval(x) if "@" not in d])
    for e in evidences_list:
        df[e] = df["binary_evidences"].apply(lambda x: 1 if e in x else 0)
    df["SEX"] = df["SEX"].map({'F': 0, 'M': 1})
    df = df[feature_columns + ["PATHOLOGY"]]
    return df

In [10]:
diagnosis_df_train = data_proc(diagnosis_df_train)
diagnosis_df_train

,AGE,SEX,E_91,E_53,E_159,E_129,E_154,E_155,E_210,E_140,...,E_193,E_168,E_180,E_67,E_171,E_111,E_182,E_103,E_23,PATHOLOGY
0,18,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,URTI
1,21,1,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,HIV (initial infection)
2,19,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Pneumonia
3,34,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,URTI
4,36,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,URTI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025597,18,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Epiglottitis
1025598,28,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Epiglottitis
1025599,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Epiglottitis
1025600,26,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Epiglottitis


In [11]:
diagnosis_df_test = data_proc(diagnosis_df_test)
diagnosis_df_test

,AGE,SEX,E_91,E_53,E_159,E_129,E_154,E_155,E_210,E_140,...,E_193,E_168,E_180,E_67,E_171,E_111,E_182,E_103,E_23,PATHOLOGY
0,49,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,GERD
1,2,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Bronchitis
2,49,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,Acute dystonic reactions
3,64,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Acute laryngitis
4,70,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,URTI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134524,52,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,SLE
134525,88,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Pulmonary neoplasm
134526,29,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Boerhaave
134527,8,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,Scombroid food poisoning


## Random Forest

In [12]:
feature_dict = {}
seed = 0
for target_disease in disease_list:
  # create path for outputs
  img_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', target_disease).replace(" ", "_")
  disease_path = f'{base_path}\\output\\diseases\\{img_filename}'
  if not os.path.exists(disease_path):
    os.makedirs(disease_path)

  # create target variable
  diagnosis_df_train["is_target_disease"] = np.where(diagnosis_df_train["PATHOLOGY"]==target_disease, 1, 0)
  diagnosis_df_test["is_target_disease"] = np.where(diagnosis_df_test["PATHOLOGY"]==target_disease, 1, 0)

  # get disease proportion
  diagnosis_df_train["is_target_disease"].value_counts().plot.pie(autopct='%.2f%%', label='')
  plt.title(f"{target_disease} - Positive and Negative Proportions\n(Train Set)")
  plt.savefig(f'{disease_path}\\{img_filename}_class_perc_train.jpg', bbox_inches='tight')
  plt.clf()
  diagnosis_df_test["is_target_disease"].value_counts().plot.pie(autopct='%.2f%%', label='')
  plt.title(f"{target_disease} - Positive and Negative Proportions\n(Test Set)")
  plt.savefig(f'{disease_path}\\{img_filename}_class_perc_test.jpg', bbox_inches='tight')
  plt.clf()

  # Undersample
  X_train = diagnosis_df_train[feature_columns]
  Y_train = diagnosis_df_train["is_target_disease"]
  rus = RandomUnderSampler(random_state=0)
  X_resampled_train, Y_resampled_train = rus.fit_resample(X_train, Y_train)

  # train model
  rf_model = RandomForestClassifier(random_state=0)
  rf_model.fit(X_resampled_train, Y_resampled_train)

  # evaluate on train set
  y_pred_train = rf_model.predict(X_resampled_train)
  train_acc = metrics.accuracy_score(Y_resampled_train, y_pred_train)
  train_recall = metrics.recall_score(Y_resampled_train, y_pred_train)
  train_f1 = metrics.f1_score(Y_resampled_train, y_pred_train)
  print(f"{target_disease} balanced train set accuracy", train_acc)
  cf_matrix = metrics.confusion_matrix(Y_resampled_train, y_pred_train)
  sns.heatmap(cf_matrix, annot=True, fmt='d')
  plt.xlabel("predicted label")
  plt.ylabel("true label")
  plt.title(f"{target_disease} - Balanced Train Set Confusion Matrix")
  plt.savefig(f'{disease_path}\\{img_filename}_train_cf.jpg', bbox_inches='tight')
  plt.clf()

  # evaluate on test set
  X_test = diagnosis_df_test[feature_columns]
  Y_test = diagnosis_df_test["is_target_disease"]
  y_pred_test = rf_model.predict(X_test)
  test_acc = metrics.accuracy_score(Y_test, y_pred_test)
  test_recall = metrics.recall_score(Y_test, y_pred_test)
  test_f1 = metrics.f1_score(Y_test, y_pred_test)
  print(f"{target_disease} test set accuracy", test_acc, "\n")
  cf_matrix = metrics.confusion_matrix(Y_test, y_pred_test)
  sns.heatmap(cf_matrix, annot=True, fmt='d')
  plt.xlabel("predicted label")
  plt.ylabel("true label")
  plt.title(f"{target_disease} - Test Set Confusion Matrix")
  plt.savefig(f'{disease_path}\\{img_filename}_test_cf.jpg', bbox_inches='tight')
  plt.clf()

  # feature importance
  top_n = 10
  rf_importance = rf_model.feature_importances_
  rf_importance = pd.DataFrame({'Feature': feature_columns, 'Importance': rf_importance}).sort_values(by="Importance", ascending=False)
  rf_importance["feature_en"] = rf_importance["Feature"].map(evidences_dict)
  rf_importance = rf_importance.head(top_n).reset_index()
  sns.barplot(data=rf_importance, y="feature_en", x="Importance", orient='h', color='steelblue')
  plt.title(f"{target_disease} - Top {top_n} most relevant symptoms")
  plt.xlabel("Symptom Importance Score")
  plt.legend('')
  plt.savefig(f'{disease_path}\\{img_filename}_ftr_importance.jpg', bbox_inches='tight')
  plt.clf()

  # save model
  with open(f'{disease_path}\\{img_filename}_model.pkl','wb') as f:
    pickle.dump(rf_model, f)
  
  # save metrics
  feature_dict[target_disease] = {
    "train_positive_diagnosis": sum(Y_train),
    "train_negative_diagnosis": len(Y_train) - sum(Y_train),
    "test_positive_diagnosis": sum(Y_test),
    "test_negative_diagnosis": len(Y_test) - sum(Y_test),
    "balanced_train_set_metrics": {
      "accuracy": train_acc,
      "recall": train_recall,
      "f1": train_f1
    },
    "test_set_metrics": {
      "accuracy": test_acc,
      "recall": test_recall,
      "f1": test_f1
    },
    f"top{top_n}_relevant_symptoms": {row["feature_en"]: row["Importance"] for _, row in rf_importance.iterrows()}#list(reversed(rf_importance["feature_en"]))
  }

# save results
with open(f"{base_path}\\output\\feature_importance.json", "w") as outfile: 
    json.dump(feature_dict, outfile, indent=True)

Spontaneous pneumothorax balanced train set accuracy 0.9973430427081283
Spontaneous pneumothorax test set accuracy 0.9910874235294992 

Cluster headache balanced train set accuracy 0.9961305518169583
Cluster headache test set accuracy 0.9921429580239205 

Boerhaave balanced train set accuracy 0.9998010610079575
Boerhaave test set accuracy 0.9992789658735292 

Spontaneous rib fracture balanced train set accuracy 1.0
Spontaneous rib fracture test set accuracy 0.9999702666339599 

GERD balanced train set accuracy 0.9982870780245583
GERD test set accuracy 0.9956440618751348 

HIV (initial infection) balanced train set accuracy 0.9999827663461207
HIV (initial infection) test set accuracy 0.9997621330716797 

Anemia balanced train set accuracy 0.9997039376295273
Anemia test set accuracy 0.9990113655791688 

Viral pharyngitis balanced train set accuracy 0.9869082768242432
Viral pharyngitis test set accuracy 0.9751131726244899 

Inguinal hernia balanced train set accuracy 0.9999258710155671
In

<Figure size 640x480 with 0 Axes>

## Logistic Regression

In [13]:
feature_dict = {}
seed = 0
for target_disease in disease_list:
  # create path for outputs
  img_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', target_disease).replace(" ", "_")
  disease_path = f'{base_path}\\output\\diseases\\{img_filename}'
  if not os.path.exists(disease_path):
    os.makedirs(disease_path)

  # create target variable
  diagnosis_df_train["is_target_disease"] = np.where(diagnosis_df_train["PATHOLOGY"]==target_disease, 1, 0)
  diagnosis_df_test["is_target_disease"] = np.where(diagnosis_df_test["PATHOLOGY"]==target_disease, 1, 0)

  # Undersample
  X_train = diagnosis_df_train[feature_columns]
  Y_train = diagnosis_df_train["is_target_disease"]
  rus = RandomUnderSampler(random_state=0)
  X_resampled_train, Y_resampled_train = rus.fit_resample(X_train, Y_train)

  # train model
  logreg_model = LogisticRegression(random_state=51, solver="lbfgs", max_iter=8000, multi_class="multinomial")
  logreg_model.fit(X_resampled_train, Y_resampled_train)

  # evaluate on train set
  y_pred_train = logreg_model.predict(X_resampled_train)
  train_acc = metrics.accuracy_score(Y_resampled_train, y_pred_train)
  train_recall = metrics.recall_score(Y_resampled_train, y_pred_train)
  train_f1 = metrics.f1_score(Y_resampled_train, y_pred_train)
  print(f"{target_disease} balanced train set accuracy", train_acc)
  cf_matrix = metrics.confusion_matrix(Y_resampled_train, y_pred_train)
  sns.heatmap(cf_matrix, annot=True, fmt='d')
  plt.xlabel("predicted label")
  plt.ylabel("true label")
  plt.title(f"{target_disease} - Balanced Train Set Confusion Matrix")
  plt.savefig(f'{disease_path}\\{img_filename}_train_logreg.jpg', bbox_inches='tight')
  plt.clf()

  # evaluate on test set
  X_test = diagnosis_df_test[feature_columns]
  Y_test = diagnosis_df_test["is_target_disease"]
  y_pred_test = logreg_model.predict(X_test)
  test_acc = metrics.accuracy_score(Y_test, y_pred_test)
  test_recall = metrics.recall_score(Y_test, y_pred_test)
  test_f1 = metrics.f1_score(Y_test, y_pred_test)
  print(f"{target_disease} test set accuracy", test_acc, "\n")
  cf_matrix = metrics.confusion_matrix(Y_test, y_pred_test)
  sns.heatmap(cf_matrix, annot=True, fmt='d')
  plt.xlabel("predicted label")
  plt.ylabel("true label")
  plt.title(f"{target_disease} - Test Set Confusion Matrix")
  plt.savefig(f'{disease_path}\\{img_filename}_test_logreg.jpg', bbox_inches='tight')
  plt.clf()

  # feature importance
  top_n = 10
  logreg_importance = logreg_model.coef_[0]
  logreg_importance = pd.DataFrame({'Feature': feature_columns, 'Importance': logreg_importance}).sort_values(by="Importance", ascending=False)
  logreg_importance["feature_en"] = logreg_importance["Feature"].map(evidences_dict)
  logreg_importance = logreg_importance.head(top_n).reset_index()
  logreg_importance = logreg_importance[logreg_importance["Importance"]>0]  # get only positive indicators of disease
  sns.barplot(data=logreg_importance, y="feature_en", x="Importance", orient='h', color='steelblue')
  plt.title(f"{target_disease} - Top {top_n} most relevant symptoms")
  plt.xlabel("Symptom Importance Score")
  plt.legend('')
  plt.savefig(f'{disease_path}\\{img_filename}_ftr_importance_logreg.jpg', bbox_inches='tight')
  plt.clf()

  # save model
  with open(f'{disease_path}\\{img_filename}_logreg_model.pkl','wb') as f:
    pickle.dump(logreg_model, f)
  
  # save metrics
  feature_dict[target_disease] = {
    "train_positive_diagnosis": sum(Y_train),
    "train_negative_diagnosis": len(Y_train) - sum(Y_train),
    "test_positive_diagnosis": sum(Y_test),
    "test_negative_diagnosis": len(Y_test) - sum(Y_test),
    "balanced_train_set_metrics": {
      "accuracy": train_acc,
      "recall": train_recall,
      "f1": train_f1
    },
    "test_set_metrics": {
      "accuracy": test_acc,
      "recall": test_recall,
      "f1": test_f1
    },
    f"top{top_n}_relevant_symptoms": {row["feature_en"]: row["Importance"] for _, row in logreg_importance.iterrows()}#list(reversed(rf_importance["feature_en"]))
  }

# save results
with open(f"{base_path}\\output\\feature_importance_logreg.json", "w") as outfile: 
    json.dump(feature_dict, outfile, indent=True)

Spontaneous pneumothorax balanced train set accuracy 0.99394804172407
Spontaneous pneumothorax test set accuracy 0.9880546201934155 

Cluster headache balanced train set accuracy 0.9956258411843876
Cluster headache test set accuracy 0.9910651235049692 

Boerhaave balanced train set accuracy 0.9994363395225464
Boerhaave test set accuracy 0.9985950984546083 

Spontaneous rib fracture balanced train set accuracy 0.9997373949579832
Spontaneous rib fracture test set accuracy 1.0 

GERD balanced train set accuracy 0.9967281265637631
GERD test set accuracy 0.9944175605259833 

HIV (initial infection) balanced train set accuracy 0.9994657567297418
HIV (initial infection) test set accuracy 0.9990931323357789 

Anemia balanced train set accuracy 0.9987565380440147
Anemia test set accuracy 0.9994722327527894 

Viral pharyngitis balanced train set accuracy 0.985594237695078
Viral pharyngitis test set accuracy 0.9724074363148466 

Inguinal hernia balanced train set accuracy 0.9998023227081789
Ingui

<Figure size 640x480 with 0 Axes>